<a href="https://colab.research.google.com/github/tdiego74/Clase_Data_Science_III_PBLC/blob/main/Proyecto_DS_III_Entrega_Final_Tocci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Analisis de Reseñas de FilmAffinity**

##**Abstracto con Motivación y Audiencia**

En este proyecto se analiza un conjunto de datos de reseñas obtenidas de FilmAffinity. Este trabajo está orientado a explorar patrones en el texto, realizar análisis de sentimientos y aplicar técnicas de procesamiento de lenguaje natural (NLP) para identificar insights significativos.

El objetivo principal es demostrar el flujo completo de análisis: desde la carga y limpieza del texto hasta el desarrollo de visualizaciones y modelos predictivos que puedan clasificar las reseñas según su polaridad o calificación.


##**Contexto Comercial y Analítico**

FilmAffinity es una plataforma de recomendaciones y críticas de películas que permite a los usuarios valorar y escribir reseñas sobre títulos cinematográficos. Las reseñas de los usuarios son una fuente valiosa de información para comprender las preferencias y emociones asociadas con el consumo de contenido audiovisual.

Este análisis busca responder preguntas como: ¿Qué palabras o frases son más comunes en las reseñas positivas y negativas? ¿Cómo varían las emociones según el género cinematográfico? Este conocimiento podría ser utilizado por la industria del entretenimiento para adaptar contenido a las expectativas de los usuarios.


##**Preguntas/Hipótesis a Resolver**

1. ¿Qué palabras o frases predominan en las reseñas positivas y negativas?
2. ¿Es posible clasificar automáticamente las reseñas según su calificación o polaridad?
3. ¿Qué características textuales pueden estar asociadas con ciertas puntuaciones o géneros?


##**Objetivo**

El objetivo de este proyecto es analizar y procesar el texto de las reseñas de FilmAffinity mediante técnicas de NLP. Esto incluye:
- Preprocesamiento y limpieza del texto.
- Análisis exploratorio para identificar patrones significativos.
- Entrenamiento de un modelo para predecir la polaridad o calificación de las reseñas.



In [1]:
#Lectura del dataset

import pandas as pd

# Leer el archivo desde GitHub usando un enfoque más robusto
df = pd.read_csv(
    'https://github.com/tdiego74/Clase_Data_Science_III_PBLC/raw/main/reviews_filmaffinity.csv',
    sep=r'\|\|',  # Expresión regular para delimitador ||
    engine='python',
    quoting=3  # Ignorar comillas
)

# Mostrar las primeras filas
print("Primeras filas del dataset:")
print(df.head())

# Mostrar información del DataFrame
print("\nInformación general del dataset:")
print(df.info())





Primeras filas del dataset:
               film_name   gender film_avg_rate  review_rate  \
0  Ocho apellidos vascos  Comedia           6,0          3.0   
1  Ocho apellidos vascos  Comedia           6,0          2.0   
2  Ocho apellidos vascos  Comedia           6,0          2.0   
3  Ocho apellidos vascos  Comedia           6,0          2.0   
4  Ocho apellidos vascos  Comedia           6,0          2.0   

                                        review_title  \
0     OCHO APELLIDOS VASCOS...Y NINGÚN NOMBRE PROPIO   
1                                     El perro verde   
2  Si no eres de comer mierda... no te comas esta...   
3                                    Aida: The movie   
4               UN HOMBRE SOLO (Julio Iglesias 1987)   

                                         review_text  
0  La mayor virtud de esta película es su existen...  
1  No soy un experto cinéfilo, pero pocas veces m...  
2  Si no eres un incondicional del humor estilo T...  
3  No sé qué está pasando, si 

In [3]:
#Lectura del corpus

import numpy as np

# 1. Muestra aleatoria de reseñas
print("Muestra aleatoria de reseñas:")
print(df['review_text'].sample(5, random_state=42))

# 2. Longitud de cada reseña (número de palabras)
df['review_length'] = df['review_text'].apply(lambda x: len(str(x).split()))
print("\nLongitud promedio de las reseñas:")
print(df['review_length'].mean())

# 3. Distribución de longitud de reseñas
print("\nEstadísticas descriptivas de longitud de las reseñas:")
print(df['review_length'].describe())

# 4. Palabras más comunes (sin limpieza)
from collections import Counter

# Combinar todas las reseñas en un solo texto
all_words = ' '.join(df['review_text'].dropna())

# Contar palabras
word_counts = Counter(all_words.split())
common_words = word_counts.most_common(10)
print("\nPalabras más comunes (sin limpiar):")
for word, count in common_words:
    print(f"{word}: {count}")


Muestra aleatoria de reseñas:
7529    Con tanta promoción, markéting, buenas crítica...
8125    Vale que la primera no era lo que se dice una ...
7071    ¿Porqué no llamarlo “Los crímenes de la univer...
8572    Dirigida por Alberto Rodríguez ("Grupo 7"), el...
5494    Es una película muy original que engancha desd...
Name: review_text, dtype: object

Longitud promedio de las reseñas:
213.72579332790886

Estadísticas descriptivas de longitud de las reseñas:
count    8603.000000
mean      213.725793
std       154.962512
min         5.000000
25%       100.000000
50%       170.000000
75%       288.000000
max      1063.000000
Name: review_length, dtype: float64

Palabras más comunes (sin limpiar):
de: 95587
que: 76809
la: 59955
y: 51432
en: 39433
el: 35813
a: 34636
es: 24943
un: 24872
no: 23675


In [2]:

# Analisis Inicial

# Distribución de géneros
print("Distribución de géneros:")
print(df['gender'].value_counts())

# Distribución de calificaciones individuales (review_rate)
print("\nDistribución de calificaciones (review_rate):")
print(df['review_rate'].value_counts())

# Distribución de calificaciones promedio de películas (film_avg_rate)
print("\nEstadísticas descriptivas de calificaciones promedio (film_avg_rate):")
print(df['film_avg_rate'].describe())


Distribución de géneros:
gender
Comedia       2495
Drama         1671
Terror         941
Aventuras      939
Thriller       762
Fantástico     495
Cine negro     419
Romance        284
Animación      284
Intriga        248
Musical         65
Name: count, dtype: int64

Distribución de calificaciones (review_rate):
review_rate
8.0     1292
7.0     1282
6.0     1256
5.0      978
9.0      833
3.0      695
1.0      639
4.0      590
2.0      523
10.0     513
Name: count, dtype: int64

Estadísticas descriptivas de calificaciones promedio (film_avg_rate):
count     8603
unique      28
top        7,2
freq      1129
Name: film_avg_rate, dtype: object
